In [294]:
'''day 3'''
import numpy as np

In [295]:
### read in data
with open('data/day3.txt', 'r') as day3:
    data = [line.strip() for line in day3]

### convert to np.array
data = np.array([list(row) for row in data])

### pad so as to avoid dealing with boundary conditions
data = np.pad(data, pad_width = 1, mode = 'constant', constant_values = '.')

In [296]:
'''part 1'''
rows = len(data)
cols = len(data[0])

numbers = [str(i) for i in range(10)]
num_idx = []
symbol_idx = []
period_idx = []

### separate out the coordinates for numbers, symbols, and periods
for i in range(rows):
    for j in range(cols):
        if data[i,j] in numbers:
            num_idx.append((i,j))
        elif data[i,j] == '.':
            period_idx.append((i,j))
        else:
            symbol_idx.append((i,j))
        

assert len(num_idx) + len(symbol_idx) + len(period_idx) == rows*cols


viable_idx = []

for coord in symbol_idx:
    x, y = coord
    x_c, y_c = np.mgrid[x-1:x+2, y-1:y+2]
    
    # pair up the coordinates and add to viable idx list
    for i in range(3):
        for j in range(3):
            if (x_c[i,j], y_c[i,j]) != coord:
                viable_idx.append((x_c[i,j], y_c[i,j]))

# remove duplicates & check for overlap with num_idx
part_nums = set(viable_idx) & set(num_idx)
#print(part_nums)

######


### store row and col of first number in data
row, col = num_idx[0]

### clump numbers together
current_num = []
clumped_nums = []
for x, y in num_idx:
    if x == row:
        if y == col:
            current_num.append((x,y))
            col += 1
        else:
            if len(current_num) != 0:
                clumped_nums.append(current_num)
            current_num = [(x,y)]
            col = y + 1
    else:
        clumped_nums.append(current_num)
        current_num = [(x,y)]
        row += 1
        col = y+1
    
    if (x, y) == num_idx[-1]:
        clumped_nums.append(current_num)
######

clumped_part_nums = []
for i in clumped_nums:
    if set(i) & part_nums:
        clumped_part_nums.append(i)

final_numbers = []
for i in clumped_part_nums:
    digits = [int(data[c[0],c[1]]) for c in i]
    length = len(digits)
    digits = [10**(length-1-p) * digits[p] for p in range(length)]
    final_numbers.append(digits)
        
final_numbers = [sum(num) for num in final_numbers]

print(sum(final_numbers))

526404


In [330]:
''' part 2 '''
rows = len(data)
cols = len(data[0])

numbers = [str(i) for i in range(10)]
num_idx = []
gear_idx = []

### separate out the coordinates for numbers and gears
for i in range(rows):
    for j in range(cols):
        if data[i,j] in numbers:
            num_idx.append((i,j))
        elif data[i,j] == '*':
            gear_idx.append((i,j))

gear_hoods = []
### isolate gears with 2 numbers in neighborhood
for coord in gear_idx:
    x, y = coord
    x_c, y_c = np.mgrid[x-1:x+2, y-1:y+2]
    
    # pair up the coordinates and add to viable idx list
    gear_hoods.append(
    [(x_c[i, j], y_c[i, j]) for i in range(3) for j in range(3) if not (i == 1 and j == 1)])
    
#print(gear_hoods)


gear_nums = []
for hood in gear_hoods:
    coords = []
    x = 0
    tracker = []
    for i in range(len(hood)):
        if data[hood[i]].isnumeric():
            coords.append(hood[i])
            tracker.append(i)
            x += (len(tracker) == 1)*1
            if len(tracker) > 1:
                if tracker[-1] - tracker[-2] == 1:
                    x += (tracker[-1] in (3, 4, 5))*1
                    coords.append(hood[i])
                else:
                    x += 1
                    coords.append(hood[i])


    if x == 2:
        for r in (0, -1):
            gear_nums.append(coords[r])

### create map from single coord to clumped group
coord_to_group = {coord: group for group in clumped_nums for coord in group}

### replace individual gear digits with their clumped numbers
clumped_gear_nums = [coord_to_group[coord] for coord in gear_nums if coord in coord_to_group]


In [332]:
final_answer_lists = []
for i in range(0, len(clumped_gear_nums), 2):
    # Process the first number
    num1_str = ''.join([str(int(data[x, y])) for x, y in clumped_gear_nums[i]])
    num1 = int(num1_str)

    # Process the second number
    num2_str = ''.join([str(int(data[x, y])) for x, y in clumped_gear_nums[i + 1]])
    num2 = int(num2_str)

    # Multiply and store the result
    final_answer_lists.append(num1 * num2)

In [299]:
final_answer_list = []
unique_gear_tuples = set(tuple(x) for x in clumped_gear_nums)
unique_lists = [list(x) for x in unique_gear_tuples]
for num in unique_lists:
    
    final_answer_list.append(num[0][0] * num[0][1])
print(sum(final_answer_list)) 

for i in unique_lists:
    digits = [int(data[c[0],c[1]]) for c in i]
    length = len(digits)
    digits = [10**(length-1-p) * digits[p] for p in range(length)]
    final_answer_lists.append(digits)

# 952782 too low, 1341917 wrong
# 1326411
# 1925681
# 1879775 wrong

3102515


In [266]:
''' generate colored data html for visualization'''
flattened_list = [tuple for sublist in clumped_gear_nums for tuple in sublist]

def generate_html_with_red_coords(data, coords):
    html_output = "<html><body><pre>"
    for i, row in enumerate(data):
        for j, char in enumerate(row):
            # Check if the current coordinate is in the list
            if (i, j) in coords:
                # If so, wrap the character in a span with red color
                html_output += f"<span style='color:red;'>{char}</span>"
            else:
                html_output += char
        html_output += "\n"
    html_output += "</pre></body></html>"
    return html_output

# Generate HTML representation
html_representation = generate_html_with_red_coords(data, gear_nums)

# Displaying a part of the HTML representation for review
output_file_path = 'data/day_3_colored_output.html'

# Saving the HTML representation to a file
with open(output_file_path, 'w') as file:
    file.write(html_representation)

In [334]:
print(sum(final_answer_lists))

84399773


In [1]:
# ## someone elxes code
# with open('data/day3.txt', 'r') as f:
#     engine = ['.{}.'.format(row) for row in f.read().split('\n')]

# def get_adjacent(r, c):
#     part_numbers = set()
#     offsets = ((-1, -1), (-1, 0), (-1, 1), (0, -1), 
#                (0, 1), (1, -1), (1, 0), (1, 1))            
#     for x, y in offsets:
#         if engine[r + x][c + y].isdigit():
#             left_pos = right_pos = c + y
#             while engine[r + x][left_pos - 1].isdigit():
#                 left_pos -= 1
#             while engine[r + x][right_pos + 1].isdigit():
#                 right_pos += 1
#             part_numbers.add(int(engine[r + x][left_pos: right_pos + 1]))
#     return part_numbers

# def parts_list():
#     all_parts = []
#     for r, row in enumerate(engine):
#         for c, symbol in enumerate(row):
#             if not symbol.isdigit() and symbol != '.':
#                 all_parts.append((symbol, get_adjacent(r, c)))
#     return all_parts

# def AOC_2023_day3_pt1():
#     return sum(sum(nums) for _, nums in parts_list())

# def AOC_2023_day3_pt2():
#     total = 0
#     for symbol, nums in parts_list():
#         if symbol == '*' and len(nums) == 2:
#             total += nums.pop() * nums.pop()
#     return total

# print(AOC_2023_day3_pt1())
# print(AOC_2023_day3_pt2())

526404
84399773
